<div style="background:#0f62fe;color:#fff;padding:14px 16px;border-radius:8px;margin:8px 0;font-weight:700;">
Module 5.1 — Sampler Primitive Options
</div>

**Learning Objectives**

- Identify what `SamplerV2` returns and where options are set.
- Set default options on `sampler.options` and override per `run()`.
- Control **shots** and read the result object.
- Configure **Dynamical Decoupling (DD)** and **Pauli Twirling** (backend-dependent).
- Validate settings and understand simulator vs. hardware behavior.



---



<div style="background:#001d6c;color:#fff;padding:10px 12px;border-radius:6px;margin:8px 0;font-weight:600;">
5.1.1 — Sampler overview & where options live
</div>

**Overview**  
- `SamplerV2` returns **shot-based bitstring distributions** (probabilities/counts + metadata).
- **Defaults** live on `sampler.options`; per-execution overrides go in `run(...)`.
**Tip:** Construct → set defaults → `run()` with PUBs → inspect `.result()`.

**Example Code: Basics**

In [1]:
# Explanation: minimal pattern (requires qiskit & qiskit-ibm-runtime)
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import SamplerV2

qc = QuantumCircuit(2)
qc.h(0); qc.cx(0,1); qc.measure_all()

sampler = SamplerV2()
print('Default options (truncated):', sampler.options)

pubs = [qc]
job = sampler.run(pubs)
res = job.result()
print('Result[0] distribution-like view (API shape may vary):', res[0])

ValueError: A backend or session must be specified.



---



<div style="background:#001d6c;color:#fff;padding:10px 12px;border-radius:6px;margin:8px 0;font-weight:600;">
5.1.2 — Setting & overriding options (workflow)
</div>

**Workflow**  
1) Choose backend/session  →  2) `SamplerV2()`  →  3) set `sampler.options`  →  4) override selectively in `run()`.

**Example Code: Defaults vs. Per‑run Override**

In [ ]:
from qiskit_ibm_runtime import SamplerV2
sampler = SamplerV2()
# Set defaults on the live sampler
sampler.options.default_shots = 2048
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = 'XY4'
sampler.options.twirling.enable = True

# Per-job override (temporarily disable twirling for this run)
job = sampler.run([qc], shots=1024, options={'twirling': {'enable': False}})
print('Job submitted with per-run override; check results to confirm.')



---



<div style="background:#001d6c;color:#fff;padding:10px 12px;border-radius:6px;margin:8px 0;font-weight:600;">
5.1.3 — Shots & measurement return
</div>

- Control shots globally (`sampler.options.default_shots`) or per call (`run(shots=...)`).
- Results expose counts/probabilities keyed by bitstrings plus metadata.
**Exam focus:** how to change shot count; how to read the result object.

**Example Code: Reading Results**

In [ ]:
job = sampler.run([qc], shots=4096)
result = job.result()
try:
    dist0 = result[0]
    print('Top items:', list(dist0.items())[:5])
except Exception as e:
    print('Result shape differs by version:', e)



---



<div style="background:#001d6c;color:#fff;padding:10px 12px;border-radius:6px;margin:8px 0;font-weight:600;">
5.1.4 — Dynamical decoupling (DD)
</div>

**Concept**  
DD inserts idle-time pulses to mitigate dephasing. Enable via `sampler.options.dynamical_decoupling.enable=True` and select a sequence (e.g., `XY4`). **Backend support required.**

**Example Code: Enable DD**

In [ ]:
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = 'XY4'
print('DD enabled with XY4 (if backend supports).')



---



<div style="background:#001d6c;color:#fff;padding:10px 12px;border-radius:6px;margin:8px 0;font-weight:600;">
5.1.5 — Twirling
</div>

**Concept**  
Pauli twirling randomizes coherent errors, making them effectively stochastic and easier to mitigate. Often **combinable with DD**, depending on backend.

**Example Code: Enable Twirling**

In [ ]:
sampler.options.twirling.enable = True
print('Twirling enabled (subject to backend support).')



---



<div style="background:#001d6c;color:#fff;padding:10px 12px;border-radius:6px;margin:8px 0;font-weight:600;">
5.1.6 — Backend/simulator notes & quick validation
</div>

- Many options (DD, twirling) impact **hardware** runs; local ideal simulators may not reflect changes.
- To simulate realistically, mirror target device noise/coupling.
- **Validation idea:** run with/without DD/twirling and compare distributions.

**Example Code: A/B Toggle Scaffold**

In [ ]:
def run_with_options(enable_dd, enable_twirling, shots=2048):
    sampler.options.dynamical_decoupling.enable = enable_dd
    sampler.options.twirling.enable = enable_twirling
    job = sampler.run([qc], shots=shots)
    return job.result()
# r0 = run_with_options(False, False)
# r1 = run_with_options(True, False)
# r2 = run_with_options(False, True)
# r3 = run_with_options(True, True)
print('Toggle sets prepared; uncomment to compare distributions.')



---



<div style="background:#001d6c;color:#fff;padding:10px 12px;border-radius:6px;margin:8px 0;font-weight:600;">
5.1.7 — Sample Exam‑Style Questions
</div>

Provide brief context before each question, as in the client template. Answers are listed after the questions.



**Q1 (5.1.1)** In which place do default sampler options live?  
A. PUB only  
B. `sampler.options`  
C. Backend config  
D. `run()` only

**Q2 (5.1.1)** What does the sampler conceptually return?  
A. Statevector  
B. Eigenvalues  
C. Bitstring distribution  
D. Unitary

**Q3 (5.1.2)** Correct workflow?  
A. run() → set options → backend  
B. backend → sampler → set defaults → run()  
C. set defaults → sampler → run()  
D. sampler → run() → set defaults

**Q4 (5.1.2)** How to tweak behavior for one job?  
A. Edit calibrations  
B. Pass overrides in `run(...)`  
C. Reinstall  
D. Not possible

**Q5 (5.1.3)** Change shots for a single job via:  
A. `sampler.options.default_shots`  
B. `run(shots=...)`  
C. `sampler.options.time_limit`  
D. PUB parameters

**Q6 (5.1.3)** The result object typically exposes:  
A. Unitary  
B. Counts/probabilities  
C. Bloch vectors  
D. Cal data only

**Q7 (5.1.4)** DD primarily mitigates:  
A. Sampling variance  
B. Idle-time dephasing  
C. Queue wait  
D. Transpilation time

**Q8 (5.1.4)** A common DD sequence:  
A. HZH  
B. XY4  
C. T-echo  
D. SWAP3

**Q9 (5.1.5)** Twirling is for:  
A. Increase crosstalk  
B. Randomize coherent errors  
C. Remove readout errors  
D. Double shots

**Q10 (5.1.5)** DD and twirling can be:  
A. Mutually exclusive  
B. Combined (backend-dependent)  
C. Simulator-only  
D. Stabilizer-only

**Q11 (5.1.6)** True statement:  
A. DD/twirling always visible on ideal sim  
B. Often require hardware support  
C. Shots ignored on hardware  
D. Noise cannot be mirrored

**Q12 (5.1.6)** Quick validation:  
A. Compare runs with/without DD/twirling  
B. Check CPU temp  
C. Change optimizer  
D. Alter qubit frequency

**Answers:** 1‑B, 2‑C, 3‑B, 4‑B, 5‑B, 6‑B, 7‑B, 8‑B, 9‑B, 10‑B, 11‑B, 12‑A




---



**Summary**  
Default vs per‑run options, shots & results, DD & twirling concepts, and practical validation on hardware vs. simulator.